In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
from nltk.corpus import stopwords
df = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')

df = df[['Title','Genre','Director','Actors','Plot']]
wnlm = WordNetLemmatizer()
def text_preprocessor(text):
  transformed_text = re.sub('[^a-zA-Z]',' ',str(text))        #Remove all characters which are not alphabets
  transformed_text = transformed_text.lower()                 #Convert all alphabets to lower case to avoid adding unnecessary complexities
  transformed_text = transformed_text.split()                 #Split the obtained sentence into words
  transformed_text = [wnlm.lemmatize(word) for word in transformed_text if word not in set(stopwords.words('english'))]
  transformed_text = ' '.join(transformed_text)
  return transformed_text
tags = []
for plot in df['Plot']:
  transformed_plot = text_preprocessor(plot)
  tags.append(transformed_plot)
df['Plot'] = tags
directors = []
for di in df['Director']:
  directors.append(text_preprocessor(di))
df['Director'] = directors
actors = []
for a in df['Actors']:
  actors.append(text_preprocessor(a))
df['Actors'] = actors
genres = []
for g in df['Genre']:
  genres.append(text_preprocessor(g))
df['Genre'] = genres




print("L",df.shape)
df['merged'] = df['Genre']+" " + df['Director']+" "+df['Actors']+" " + df['Plot']
df.head()




In [ ]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df['merged'])
cosine_similarity_matrix = cosine_similarity(count_matrix, count_matrix)
print("C",cosine_similarity_matrix)

In [ ]:
indices = pd.Series(df.index)
print(indices)
def recommend_movies(title):
  recommended_movies = []
  id = 0
  for i in range(len(df['Title'])):
    if df['Title'][i] == title:
      id = i
      break

    # creating a Series with the similarity scores in descending order
  score_series = pd.Series(cosine_similarity_matrix[id]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
  top_10_indexes = list(score_series.iloc[1:11].index)

    # populating the list with the titles of the best 10 matching movies
  for i in top_10_indexes:
      recommended_movies.append(list(df.index)[i])
  sol = []
  for i in recommended_movies:
    sol.append(df['Title'][i])

  return sol
recommend_movies('Fargo')